<a href="https://colab.research.google.com/github/GMayumi/FreeCodeCamp/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Reading data
train_dataframe = pd.read_csv(test_file_path, sep="\t", header=None, names=["classification", "text"])
test_dataframe = pd.read_csv(test_file_path, sep="\t", header=None, names=["classification", "text"])

In [ ]:
# Convert categorical data to numbers
print("classification: ", train_dataframe["classification"].unique())

train_dataframe["classification"] = pd.factorize(train_dataframe["classification"])[0]
test_dataframe["classification"] = pd.factorize(test_dataframe["classification"])[0]

print("classification: ", train_dataframe["classification"].unique())

In [ ]:
# Create labels and convert to TensorFlow datasets
train_labels =  train_dataframe["classification"].values
train_ds = tf.data.Dataset.from_tensor_slices((train_dataframe["text"].values, train_labels))

test_labels =  test_dataframe["classification"].values
test_ds = tf.data.Dataset.from_tensor_slices((test_dataframe["text"].values, test_labels))

In [ ]:
# Adjusting datasets for performance
BUFFER_SIZE = 100
BATCH_SIZE = 32
train_ds = train_ds.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
# Define and adapt text vectorization layer
vectorize_layer = tf.keras.layers.TextVectorization(
    output_mode='int',
    max_tokens=1000,
    output_sequence_length=1000,
)

vectorize_layer.adapt(train_ds.map(lambda text, label: text))

In [ ]:
# Creating model
model = tf.keras.Sequential()

# maps text features to integer sequences
model.add(vectorize_layer)

# transforms categorical data into dense, continuous vector spaces where similar items are closer together
model.add(tf.keras.layers.Embedding(len(vectorize_layer.get_vocabulary()), 64, mask_zero=True))

# processes sequential data in both forward and backward directions (capture both past and future context of the input sequence)
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)))

model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(from_logits=True), # final layer without SoftMax/Sigmoid layer
    optimizer = 'adam',
    metrics = ['accuracy'],
)

model.summary()

In [ ]:
# Train model
history = model.fit(
    train_ds,
    validation_data = test_ds,
    validation_steps = 30,
    epochs = 10
)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    ps = model.predict(np.array([pred_text]).astype("object"))
    return [ps[0][0], "ham" if ps[0][0] < 0.5 else "spam"]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
